# TAREA 2
## Modelos de lenguage: N-gramas
### Parte 2

Para el desarrollo de los puntos 5 y 6, se va a proceder con la implementación del cálculo de la perplejidad para los modelos unigrama, bigrama y trigrama. Se va a utilizar los archivos de prueba generados en el punto 3 (20N_4_testing.txt y BAC_4_testing.txt) y las probabilidades ya generadas en el punto 4 (archivos de unigramas, bigramas y trigramas).

A continuación se realizan las importaciones necesarias para importar bibliotecas para el procesamiento de lenguaje natural con NLTK, manejo de archivos comprimidos, web scraping, manipulación de datos JSON y combinaciones, además de optimización de cálculos usando GPU con cupy. Facilita tareas como tokenización, creación de n-gramas y análisis eficiente de datos textuales.

In [22]:
import os
import re
from collections import Counter
import nltk
from nltk import (
sent_tokenize,
word_tokenize,
ngrams,
bigrams,
trigrams
)
import tarfile
import lxml
from bs4 import BeautifulSoup
import requests
import random
import zipfile
import json
from itertools import combinations_with_replacement
import cupy as cp
import math

Se define la ruta para leer los resultados de la implementación del punto 3 y 4 con los archivos de prueba y los modelos N-gram generados

In [9]:
resultados_dir = "C:/Users/LENOVO/Downloads/Datasets/resultados"  # Ruta al directorio con los resultados de los puntos anteriores

Las siguientes líneas definen las rutas de los archivos de prueba generados en el punto 3. path_testing_BAC y path_testing_20N apuntan a los archivos con las oraciones de prueba del 20% de los datos, que se utilizarán para calcular la perplejidad de los modelos N-Gram.

In [10]:
path_testing_BAC = os.path.join(resultados_dir, 'BAC_4_testing.txt')
path_testing_20N = os.path.join(resultados_dir, '20N_4_testing.txt')


Las siguientes , definen las rutas de los archivos JSON que contienen los modelos N-Gram (unigrama, bigrama y trigram) para los conjuntos de datos BAC y 20N, ubicados en la carpeta resultados.

In [11]:
unigram_file_BAC = os.path.join(resultados_dir, 'BAC_4_unigrams.json')
bigram_file_BAC = os.path.join(resultados_dir, 'BAC_4_bigrams.json')
trigram_file_BAC = os.path.join(resultados_dir, 'BAC_4_trigrams.json')

unigram_file_20N = os.path.join(resultados_dir, '20N_4_unigrams.json')
bigram_file_20N = os.path.join(resultados_dir, '20N_4_bigrams.json')
trigram_file_20N = os.path.join(resultados_dir, '20N_4_trigrams.json')

### 5. (15p) Using the test dataset, calculate the perplexity of each of the language models. Report the results obtained.If you experience variable overflow, use probabilities in log space

A continuación, la siguiente implementación calcula la perplejidada para un modelo de unigramas. Carga el modelo previamente generado, y para cada palabra en las oraciones de prueba, obtiene su probabilidad (o usa la de <UNK> si no está en el modelo). Suma los logaritmos de esas probabilidades y luego calcula la perplejidad total a partir de la suma acumulada.

In [12]:
def calculate_perplexity_unigram(test_file, model_file):
    # Cargar el modelo de unigramas
    with open(model_file, 'r') as f:
        unigram_model = json.load(f)
    
    # Leer el archivo de prueba
    with open(test_file, 'r', encoding='utf-8') as f:
        sentences = f.readlines()

    log_prob_sum = 0
    total_words = 0

    # Calcular la probabilidad para cada palabra en el conjunto de prueba
    for sentence in sentences:
        words = sentence.strip().split()
        for word in words:
            prob = unigram_model.get(word, unigram_model.get('<UNK>', 1e-6))  # Si no está, usar <UNK>
            log_prob_sum += math.log(prob)
            total_words += 1
    
    # Calcular la perplexity
    perplexity = math.exp(-log_prob_sum / total_words)
    return perplexity

perplexity_unigram_BAC = calculate_perplexity_unigram(path_testing_BAC, unigram_file_BAC)
perplexity_unigram_20N = calculate_perplexity_unigram(path_testing_20N, unigram_file_20N)
print("Perplejidad BAC (unigrama):", perplexity_unigram_BAC)
print("Perplejidad 20N (unigrama):", perplexity_unigram_20N)

Perplexity BAC (unigrama): 605.4393675378017
Perplexity 20N (unigrama): 745.3769679173222



Esta función calcula la perplejidad para un modelo de bigramas. Para cada bigrama en las oraciones de prueba, obtiene su probabilidad del modelo (o usa <UNK> si no está en el modelo). Luego, acumula los logaritmos de las probabilidades para calcular la perplejidad total.

In [13]:
def calculate_perplexity_bigram(test_file, model_file):
    # Cargar el modelo de bigramas
    with open(model_file, 'r') as f:
        bigram_model = json.load(f)
    
    # Leer el archivo de prueba
    with open(test_file, 'r', encoding='utf-8') as f:
        sentences = f.readlines()

    log_prob_sum = 0
    total_bigrams = 0

    # Calcular la probabilidad para cada bigrama en el conjunto de prueba
    for sentence in sentences:
        words = sentence.strip().split()
        bigrams_seq = list(bigrams(words))
        for bg in bigrams_seq:
            prob = bigram_model.get(str(bg), bigram_model.get(str(('<UNK>', bg[1])), 1e-6))  # Si no está, usar <UNK>
            log_prob_sum += math.log(prob)
            total_bigrams += 1
    
    # Calcular la perplexity
    perplexity = math.exp(-log_prob_sum / total_bigrams)
    return perplexity

perplexity_bigram_BAC = calculate_perplexity_bigram(path_testing_BAC, bigram_file_BAC)
perplexity_bigram_20N = calculate_perplexity_bigram(path_testing_20N, bigram_file_20N)
print("Perplejidad BAC (bigrama):", perplexity_bigram_BAC)
print("Perplejiad 20N (bigrama):", perplexity_bigram_20N)

Perplexity BAC (bigrama): 2746.495730388989
Perplexity 20N (bigrama): 4944.588159592507


Esta función calcula la perplejidad para un modelo de trigramas. Obtiene la probabilidad de cada trigrama en las oraciones de prueba y, si no está presente, utiliza <UNK>. Luego, acumula los logaritmos de las probabilidades para calcular la perplejidad total.

In [14]:
def calculate_perplexity_trigram(test_file, model_file):
    # Cargar el modelo de trigramas
    with open(model_file, 'r') as f:
        trigram_model = json.load(f)
    
    # Leer el archivo de prueba
    with open(test_file, 'r', encoding='utf-8') as f:
        sentences = f.readlines()

    log_prob_sum = 0
    total_trigrams = 0

    # Calcular la probabilidad para cada trigrama en el conjunto de prueba
    for sentence in sentences:
        words = sentence.strip().split()
        trigrams_seq = list(trigrams(words))
        for tg in trigrams_seq:
            prob = trigram_model.get(str(tg), trigram_model.get(str(('<UNK>', tg[1], tg[2])), 1e-6))  # Si no está, usar <UNK>
            log_prob_sum += math.log(prob)
            total_trigrams += 1
    
    # Calcular la perplexity
    perplexity = math.exp(-log_prob_sum / total_trigrams)
    return perplexity

perplexity_trigram_BAC = calculate_perplexity_trigram(path_testing_BAC, trigram_file_BAC)
perplexity_trigram_20N = calculate_perplexity_trigram(path_testing_20N, trigram_file_20N)
print("Perplejidad BAC (trigrama):", perplexity_trigram_BAC)
print("Perplejidad 20N (trigrama):", perplexity_trigram_20N)

Perplexity BAC (trigrama): 43353.28489503441
Perplexity 20N (trigrama): 58320.094203029264


### 6. (15p) Using your best language model, build a method/function that automatically generates sentences by receiving the first word of a sentence as input. Take different tests and document them.

A continuación, se efectúa la implementación para generar oraciones usando modelos de N-Gramas (unigrama, bigrama o trigramas). Dependiendo del valor de n, la función selecciona la siguiente palabra basándose en el contexto actual (las últimas n-1 palabras) y en las probabilidades del modelo.
Dentro de esta se maneja:
1. Se realiza la carga el modelo N-Gram desde un archivo JSON.
2. Se genera la oración usando las palabras iniciales y selecciona las siguientes según las probabilidades del modelo.

Esta implementación es funcional para diferentes modelos de N-Gramas (cambiando el valor de n), esta tiene en cuenta un control de errores para evitar problemas con el tamaño del N-Gram.

In [41]:
def generate_sentence_unigram(model_file, max_length=5):
    with open(model_file, 'r') as f:
        unigram_model = json.load(f)

    sentence = []

    # Asegurarse de que la primera palabra no sea <s> o </s>
    first_word = random.choices(
        [word for word in unigram_model.keys() if word not in ['<s>', '</s>']],
        weights=[unigram_model[word] for word in unigram_model.keys() if word not in ['<s>', '</s>']]
    )[0]
    sentence.append(first_word)

    # Continuar generando el resto de la oración
    for _ in range(max_length - 1):  
        next_word = random.choices(list(unigram_model.keys()), weights=list(unigram_model.values()))[0]
        if next_word == '</s>':
            break
        sentence.append(next_word)
    
    if len(sentence) < max_length:
        sentence.append('</s>')
    
    return ' '.join(sentence)

In [43]:
def generate_sentence_ngram(model_file, start_words, n, max_length=5):
    with open(model_file, 'r') as f:
        ngram_model = json.load(f)
    
    sentence = start_words[:]  # Comenzar con las palabras iniciales
    current_context = tuple(start_words)  # Inicialmente, el contexto será el bigrama o trigrama dado
    
    for _ in range(max_length - len(start_words)):  # Se resta la longitud inicial del start_words
        possible_next_words = [eval(k)[n-1] for k in ngram_model.keys() if len(eval(k)) >= n and eval(k)[:n-1] == current_context]        
        if not possible_next_words:  # Si no hay posibles palabras siguientes, se detiene
            break
    
        next_word = random.choices(possible_next_words, weights=[ngram_model[str((*current_context, w))] for w in possible_next_words])[0]
        
        if next_word == '</s>':  
            break
        
        sentence.append(next_word)
        
        current_context = tuple(sentence[-(n-1):])  # Mantener las últimas n-1 palabras en el contexto
    
    if len(sentence) >= max_length:
        sentence.append('</s>')

    return ' '.join(sentence)

# Ejemplo de uso
start_word_bigram = ["somebody"]
start_words_trigram = ["when", "somebody"]

generated_sentence_unigram_BAC = generate_sentence_unigram(unigram_file_BAC, max_length=5)
generated_sentence_unigram_20N = generate_sentence_unigram(unigram_file_20N, max_length=5)

generated_sentence_bigram_BAC = generate_sentence_ngram(bigram_file_BAC, start_word_bigram, n=2, max_length=5)
generated_sentence_bigram_20N = generate_sentence_ngram(bigram_file_20N, start_word_bigram, n=2, max_length=5)

generated_sentence_trigram_BAC = generate_sentence_ngram(trigram_file_BAC, start_words_trigram, n=3, max_length=5)
generated_sentence_trigram_20N = generate_sentence_ngram(trigram_file_20N, start_words_trigram, n=3, max_length=5)

print("Oración generada en unigrama BAC:", generated_sentence_unigram_BAC)
print("Oración generada en unigrama 20N:", generated_sentence_unigram_20N)
print("Oración generada en bigrama BAC:", generated_sentence_bigram_BAC)
print("Oración generada en bigrama 20N:", generated_sentence_bigram_20N)
print("Oración generada en trigrama BAC:", generated_sentence_trigram_BAC)
print("Oración generada en trigrama 20N:", generated_sentence_trigram_20N)


Oración generada en unigrama BAC: table true divorce the and
Oración generada en unigrama 20N: buf t ax NUM of
Oración generada en bigrama BAC: somebody new tricks
Oración generada en bigrama 20N: somebody stankowitz and plausible
Oración generada en trigrama BAC: when somebody messes with one </s>
Oración generada en trigrama 20N: when somebody gets offed by </s>
